In [1]:
from torchvision.datasets import FashionMNIST
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 从"./dataset/"目录加载FashionMNIST数据集，如果没有则会自动下载。
train_data = FashionMNIST(root='./dataset/', train=True, transform=transforms.ToTensor(), download=True)
test_data = FashionMNIST(root='./dataset/', train=False, transform=transforms.ToTensor(), download=True)
train_batch = DataLoader(dataset=train_data, batch_size=256,  shuffle=True,  drop_last=False)

D:\Python\Python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
class Model(torch.nn.Module):
    def __init__(self,in_features,out_features):
        super().__init__()
        self.bn = torch.nn.BatchNorm2d(1)
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.pool1 = torch.nn.MaxPool2d(2) # 13
        self.conv2 = torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3) 
        self.pool2 = torch.nn.MaxPool2d(2) # 5
        self.linear1 = torch.nn.Linear(16*5*5, 128)
        self.output = torch.nn.Linear(128, out_features, bias=False)
        
    def forward(self,x):
        x = self.bn(x)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = torch.relu(x.view(x.shape[0], -1))
        x = torch.relu(self.linear1(x))
        x = self.output(x)
        x = F.log_softmax(x,dim=1)
        return x

In [4]:
from torch.optim import Adam
from torch.nn import functional as F

# 设置随机种子，确保实验可重复性
torch.random.manual_seed(420)

# 初始化一个模型，输入图片通道数为1，输出特征为10
model = Model(in_features=1, out_features=10).to(device)
# 使用负对数似然损失函数
criterion = torch.nn.NLLLoss()
# 初始化Adam优化器，设定学习率为0.005
opt = Adam(model.parameters(), lr=0.005)


In [5]:
# 进行9次迭代
for _ in range(9):
    # 遍历数据批次
    for batch in train_batch:
        # 将输入数据X调整形状并输入到模型
        X = batch[0].to(device)
        # y为真实标签
        y = batch[1].to(device)

        # 前向传播，获取模型输出
        sigma = model.forward(X)
        # 计算损失
        loss = criterion(sigma, y)
        # 计算预测的标签
        y_hat = torch.max(sigma, dim=1)[1]
        # 计算预测正确的数量
        correct_count = torch.sum(y_hat == y)
        # 计算准确率
        accuracy = correct_count / len(y) * 100
        # 反向传播，计算梯度
        loss.backward()
        # 更新模型参数
        opt.step()
        # 清除之前的梯度
        model.zero_grad()
    # 打印当前批次的损失和准确率
    print('loss:', loss.item(), 'accuracy:', accuracy.item())

D:\Python\Python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


loss: 0.4118366539478302 accuracy: 83.33333587646484
loss: 0.2345355600118637 accuracy: 91.66667175292969
loss: 0.20766907930374146 accuracy: 92.70833587646484
loss: 0.265121728181839 accuracy: 90.625
loss: 0.18941356241703033 accuracy: 91.66667175292969
loss: 0.1682918667793274 accuracy: 94.79167175292969
loss: 0.17323146760463715 accuracy: 93.75
loss: 0.14355356991291046 accuracy: 92.70833587646484
loss: 0.3490566313266754 accuracy: 87.5


In [6]:
test_X = test_data.data.unsqueeze(dim=1).to(device)
test_y = test_data.targets.to(device)
sigma = model.forward(torch.tensor(test_X,dtype=torch.float32))
y_hat = torch.max(sigma, dim=1)[1]
correct_count = torch.sum(y_hat == test_y)
accuracy = correct_count / 10000 * 100
print('accuracy:', accuracy.item())

accuracy: 89.8499984741211


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7188\3480106166.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sigma = model.forward(torch.tensor(test_X,dtype=torch.float32))


In [7]:
test_data.data.unsqueeze(dim=1).shape

torch.Size([10000, 1, 28, 28])